In [1]:
import glob
import os
import pickle
import warnings
from pathlib import Path

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm


In [2]:
class Config:
    N_FOLD = 5
    RANDOM_SATE = 42


NB = "exp1015"

In [3]:
ROOT_DIR = Path("/kaggle")
DATA_DIR = ROOT_DIR / Path("input/atmaCup#18_dataset")

In [4]:
train_df = pl.read_csv(DATA_DIR / "train_features.csv")
test_df = pl.read_csv(DATA_DIR / "test_features.csv")

## 特徴量生成


In [5]:
# train_df と test_dfを結合（特徴量エンジニアリングをしやすくするため）
_all_df = pl.concat([train_df, test_df], how="diagonal")

In [6]:
agg_cols = [
    "vEgo",
    "aEgo",
    "steeringAngleDeg",
    "steeringTorque",
    "gas",
]  # 同一シーンから集計する値のカラム名

# 同一シーンから特徴量作成
exprs = []
exprs += [
    pl.col(agg_col).shift(-1).over("scene").alias(f"{agg_col}_shift-1") for agg_col in agg_cols
]  # 1ステップ前の時間の値
exprs += [
    pl.col(agg_col).shift(1).over("scene").alias(f"{agg_col}_shift1") for agg_col in agg_cols
]  # 1ステップ後の時間の値
exprs += [
    pl.col(agg_col).diff(-1).over("scene").alias(f"{agg_col}_diff-1") for agg_col in agg_cols
]  # 1ステップ前の時間の値との差分
exprs += [
    pl.col(agg_col).diff(1).over("scene").alias(f"{agg_col}_diff1") for agg_col in agg_cols
]  # 1ステップ後の時間の値との差分
exprs += [pl.col(agg_col).mean().over("scene").alias(f"{agg_col}_mean") for agg_col in agg_cols]  # 同一シーンの平均値
exprs += [pl.col(agg_col).std().over("scene").alias(f"{agg_col}_std") for agg_col in agg_cols]  # 同一シーンの標準偏差
exprs += [pl.col(agg_col).max().over("scene").alias(f"{agg_col}_max") for agg_col in agg_cols]  # 同一シーンの最大値
exprs += [pl.col(agg_col).min().over("scene").alias(f"{agg_col}_min") for agg_col in agg_cols]  # 同一シーンの最小値

_all_df = (
    _all_df.with_columns(
        # ID からシーンとデシ秒を作成
        pl.col("ID").str.split("_").list.get(0).alias("scene"),
        pl.col("ID").str.split("_").list.get(1).cast(pl.Int32).alias("decisecond"),
    )
    .sort(
        # shiftと diffが時系列順に並んでいる必要があるためシーンごとに時間軸でソート
        "scene",
        "decisecond",
    )
    .with_columns(exprs)
)

## feature and target


In [7]:
targets = [
    "x_0",
    "y_0",
    "z_0",
    "x_1",
    "y_1",
    "z_1",
    "x_2",
    "y_2",
    "z_2",
    "x_3",
    "y_3",
    "z_3",
    "x_4",
    "y_4",
    "z_4",
    "x_5",
    "y_5",
    "z_5",
]

# 使う特徴量を指定するより使わない特徴量を指定するほうが試行錯誤が楽
del_columns = targets + ["ID", "scene", "gearShifter"]

features = list(set(_all_df.columns) - set(del_columns))
features.sort()

print(features)

['aEgo', 'aEgo_diff-1', 'aEgo_diff1', 'aEgo_max', 'aEgo_mean', 'aEgo_min', 'aEgo_shift-1', 'aEgo_shift1', 'aEgo_std', 'brake', 'brakePressed', 'decisecond', 'gas', 'gasPressed', 'gas_diff-1', 'gas_diff1', 'gas_max', 'gas_mean', 'gas_min', 'gas_shift-1', 'gas_shift1', 'gas_std', 'leftBlinker', 'rightBlinker', 'steeringAngleDeg', 'steeringAngleDeg_diff-1', 'steeringAngleDeg_diff1', 'steeringAngleDeg_max', 'steeringAngleDeg_mean', 'steeringAngleDeg_min', 'steeringAngleDeg_shift-1', 'steeringAngleDeg_shift1', 'steeringAngleDeg_std', 'steeringTorque', 'steeringTorque_diff-1', 'steeringTorque_diff1', 'steeringTorque_max', 'steeringTorque_mean', 'steeringTorque_min', 'steeringTorque_shift-1', 'steeringTorque_shift1', 'steeringTorque_std', 'vEgo', 'vEgo_diff-1', 'vEgo_diff1', 'vEgo_max', 'vEgo_mean', 'vEgo_min', 'vEgo_shift-1', 'vEgo_shift1', 'vEgo_std']


In [8]:
# MAEを計算
def evaluation(true_values, pred_values):
    abs_diff = abs(true_values - pred_values)
    mae = np.mean(
        abs_diff.reshape(
            -1,
        )
    )
    return mae

### encoding


In [9]:
# label encdoding
categorical_columns = ["gearShifter"]

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    _all_df = _all_df.with_columns(pl.Series(le.fit_transform(_all_df[col])).alias(f"{col}_le"))
cate_features = [f"{col}_le" for c in categorical_columns]
features = list(set(features) | set(cate_features))

# count encoding
count_enc = ["gearShifter"]
_all_df = _all_df.with_columns([pl.col(c).count().over(c).alias(f"{c}_count") for c in count_enc])
count_features = [f"{c}_count" for c in count_enc]
features = list(set(features) | set(count_features))


train_df = train_df.join(_all_df, how="left", on="ID")
test_df = test_df.join(_all_df, how="left", on="ID")

In [ ]:
def train_lgbm(target):
    params = {
        "boosting_type": "gbdt",
        "metric": "mae",  # 今回の評価指標がMAEを使用
        "objective": "regression",
        "n_jobs": -1,
        "seed": Config.RANDOM_SATE,
        "learning_rate": 0.01,
        # "device": "gpu"
    }

    oof_pred = np.zeros(len(train_df))
    y_pred = np.zeros(len(test_df))
    models = []
    cv_scores = {}

    gkf = GroupKFold(n_splits=Config.N_FOLD)

    for fold, (train_index, test_index) in enumerate(gkf.split(train_df, groups=train_df["scene"])):
        print(f"====== fold {fold} ======")

        # TrainとTestに分割
        x_train, x_val = train_df[train_index].select(features), train_df[test_index].select(features)
        y_train, y_val = train_df[train_index].select(target), train_df[test_index].select(target)

        test = test_df[features]

        # create Dataset
        train_set = lgb.Dataset(
            x_train.to_pandas(),
            y_train.to_pandas(),
            categorical_feature=cate_features,
            free_raw_data=False,
        )
        val_set = lgb.Dataset(
            x_val.to_pandas(),
            y_val.to_pandas(),
            categorical_feature=cate_features,
            free_raw_data=False,
        )

        # train
        model = lgb.train(
            params,
            train_set,
            valid_sets=[train_set, val_set],
            num_boost_round=10000,
            callbacks=[
                lgb.early_stopping(stopping_rounds=100, verbose=True),
                lgb.log_evaluation(500),
            ],
        )

        models.append(model)

        fold_pred = model.predict(x_val.to_pandas())

        score = evaluation(y_val.to_numpy().reshape(-1), fold_pred)
        cv_scores[f"cv{fold}"] = score

        oof_pred[test_index] = fold_pred

        y_pred += model.predict(test.to_pandas()) / Config.N_FOLD

        print(f"cv score is {score}")

    oof_score = evaluation(train_df[target].to_numpy().reshape(-1), oof_pred)
    print(f"OOF score is {oof_score}")

    return oof_pred, y_pred, models

## 学習


In [ ]:
models_dict = {}
test_pred = []
oof_pred = []
for target in targets:
    print("=" * 50)
    print(f"# {target}")
    print("=" * 50)
    oof_preds_partial, y_pred_partial, models_partial = train_lgbm(target)
    models_dict[target] = models_partial
    oof_pred.append(oof_preds_partial)
    test_pred.append(y_pred_partial)

# x_0
====== fold 0 ======
Training until validation scores don't improve for 100 rounds


/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:335: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[500]	training's l1: 0.0944305	valid_1's l1: 0.101335
[1000]	training's l1: 0.0685654	valid_1's l1: 0.0753467
[1500]	training's l1: 0.0635339	valid_1's l1: 0.0716455


In [ ]:
evaluation(train_df[targets].to_numpy(), np.vstack(oof_pred).T)

0.21185031472732527

# submit ファイル作成


In [ ]:
sub_col_names = [
    "x_0",
    "y_0",
    "z_0",
    "x_1",
    "y_1",
    "z_1",
    "x_2",
    "y_2",
    "z_2",
    "x_3",
    "y_3",
    "z_3",
    "x_4",
    "y_4",
    "z_4",
    "x_5",
    "y_5",
    "z_5",
]
sub_df = pl.DataFrame(np.vstack(test_pred).T, schema=sub_col_names)
display(sub_df)
sub_df.write_csv("submission.csv")

x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1.45612,-0.050807,0.002389,3.017458,-0.117573,0.005708,4.552201,-0.195567,0.002728,5.826144,-0.198686,0.005409,7.245767,-0.127643,-0.008084,8.550774,-0.02874,-0.013318
0.93874,0.389052,-0.000527,1.741811,1.009938,-0.002193,2.371443,1.765806,-0.008336,2.813685,2.62283,-0.008497,3.51195,3.502601,-0.004901,4.238286,4.478624,0.002362
1.571503,0.016573,0.003865,3.253146,0.013525,0.008019,4.892635,0.029848,0.01301,6.30592,0.067152,0.014728,7.717493,0.13308,0.018196,9.01833,0.185266,0.020942
0.835832,0.064877,-0.001315,1.650979,0.214632,-0.007302,2.414782,0.549095,-0.011152,2.94753,0.837604,-0.017816,3.605272,1.479534,-0.019365,4.336746,2.189905,-0.018245
0.812953,0.004572,-0.001849,1.415035,0.000654,-0.004624,1.880035,0.011582,-0.017626,2.218841,-0.000622,-0.036413,2.152111,0.007862,-0.05392,1.712793,-0.019529,-0.068401
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
6.535473,0.009117,0.017769,13.803384,0.050178,0.04067,21.014647,0.129037,0.064933,28.275174,0.218875,0.075093,35.585978,0.331842,0.080312,43.006691,0.460795,0.069075
6.999905,0.000927,0.004368,14.893268,-0.011176,0.008717,22.944056,-0.028674,0.012684,31.102234,-0.067466,0.023056,39.279125,-0.098898,0.031083,47.512481,-0.141442,0.035101
7.415662,-0.001225,0.007195,15.664745,-0.020854,0.012552,23.896748,-0.057395,0.02172,32.060604,-0.122295,0.047701,40.124927,-0.199787,0.076695,48.056512,-0.255825,0.094876
